In [1]:
import os, math
import numpy as np
import pandas as pd
import seaborn as sns
import helpers
import xgboost as xgb

%matplotlib inline
import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn import svm
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import log_loss
from models import *

//anaconda/envs/cancer/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/envs/cancer/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
//anaconda/envs/cancer/lib/python3.6/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarnin

In [2]:
text, variants = helpers.get_training('./input/training_variants', './input/training_text')
train_full = variants.merge(text, how='inner', on='ID')

#this is 10% of the labeled data
text, variants = helpers.get_test('./input/training_variants', './input/training_text')
test_full = variants.merge(text, how='inner', on='ID')

print(train_full.shape)
print(test_full.shape)

//anaconda/envs/cancer/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-02 10:05:10.180545. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


(2656, 5)
(665, 5)


Use TF-IDF to Vectorize the texts， a feature selector, then SVM one-vs-all classification. 

In [3]:
pl1 = GeneSvm().fit_feature(train_full)
pl2 = TfidfSvm().fit_feature(train_full)

this is a model based on only gene&variations


//anaconda/envs/cancer/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-08-02 10:05:10.182482. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


this is a tfidf+svm model


In [4]:
X_train = np.concatenate((GeneSvm().get_feature(pl1, train_full), TfidfSvm().get_feature(pl2, train_full).todense()), axis=1)

this is a model based on only gene&variations
this is a tfidf+svm model


In [5]:
X_test = np.concatenate((GeneSvm().get_feature(pl1, test_full), TfidfSvm().get_feature(pl2, test_full).todense()), axis=1)

this is a model based on only gene&variations
this is a tfidf+svm model


In [6]:
y_train = train_full['Class']-1
y_test = test_full['Class']-1

In [7]:

params = {
        'eta': 0.03333,
        'max_depth': 5,
        'min_child_weight': 1,
        'gamma': 0,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'seed': 0,
        'silent': True
    }

watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_test, y_test), 'valid')]
model = xgb.train(params, xgb.DMatrix(X_train, y_train), 1000,  watchlist, verbose_eval=50, early_stopping_rounds=100)

[0]	train-mlogloss:2.12716	valid-mlogloss:2.14562
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:0.833213	valid-mlogloss:1.23274
[100]	train-mlogloss:0.495887	valid-mlogloss:1.0314
[150]	train-mlogloss:0.346433	valid-mlogloss:0.958461
[200]	train-mlogloss:0.25844	valid-mlogloss:0.924746
[250]	train-mlogloss:0.19896	valid-mlogloss:0.914477
[300]	train-mlogloss:0.154064	valid-mlogloss:0.909616
[350]	train-mlogloss:0.119402	valid-mlogloss:0.911574
[400]	train-mlogloss:0.093045	valid-mlogloss:0.916202
Stopping. Best iteration:
[301]	train-mlogloss:0.153318	valid-mlogloss:0.909238



In [8]:
#Evaluate
y_train_prob = model.predict(xgb.DMatrix(X_train), ntree_limit=model.best_ntree_limit)
y_test_prob = model.predict(xgb.DMatrix(X_test), ntree_limit=model.best_ntree_limit)
log_loss(y_test, y_test_prob, eps=1e-15, normalize=True, labels = list(range(9)))

0.90923832665036497

In [9]:
true_test_full = helpers.get_full_table('./input/test_variants','./input/test_text')
X_true_test = np.concatenate((GeneSvm().get_feature(pl1, true_test_full), TfidfSvm().get_feature(pl2, true_test_full).todense()), axis=1)

true_test_prob = model.predict(xgb.DMatrix(X_true_test), ntree_limit=model.best_ntree_limit)



this is a model based on only gene&variations
this is a tfidf+svm model


In [10]:
helpers.submission('./2nd_layer_data/xgboost/train_prob', y_train_prob)
helpers.submission('./2nd_layer_data/xgboost/test_prob', y_test_prob)
helpers.submission('./2nd_layer_data/xgboost/true_test_prob', true_test_prob)

In [11]:
#feature = GeneSvm().get_feature(pl1, true_test_full)
#np.save('./2nd_layer_data/gene-var/X_true_test', feature)